In [68]:
from PIL import Image, ImageChops, ImageEnhance
import numpy as np

f = Path("models/model_structure.json")
model_structure_A = f.read_text()
model_A = model_from_json(model_structure_A)
model_A.load_weights("models/model_weights.h5")

f = Path("models/model_structure-3.json")
model_structure_B = f.read_text()
model_B = model_from_json(model_structure_B)
model_B.load_weights("models/model_weights-3.h5")

def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality=quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

def predict_deepfake(image_path):
    image_size = (128, 128)
    
    ela_img = convert_to_ela_image(image_path, 90)
    img = np.array(ela_img.resize(image_size)).flatten() / 255.0
    img = np.asarray(img)
    img = img.reshape(-1, 128, 128, 3)
    
    prediction_A = model_A.predict(img)
    prediction_B = model_B.predict(img)
    
    weight_A = 0.3
    weight_B = 0.2
    
    combined_prediction = weight_A * prediction_A[0][1] + weight_B * prediction_B[0][1]
    
    if combined_prediction >= 0.5:
        return "Real"
    else:
        return "Fake"

# Usage example
image_path = "u_input/cat2.jpg"
result = predict_deepfake(image_path)
print("Prediction:", result)

1/1 [==============================] - 0s 26ms/step
Prediction: Fake
